# Processing of Slocum Glider-AD2CP Data: RU36 2023

jgradone@marine.rutgers.edu     03/10/2022    Initial <br>
jgradone@marine.rutgers.edu     06/16/2022    Update for pre-processing <br>
engdahl@marine.rutgers.edu      01/19/2024    Modify Notebook 2, 3 & 4 with new example using nbronikowski@mun.ca functions    
engdahl@marine.rutgers.edu      04/04/2024    Modified make_dataset to have magnetic declination function to rotate U and V dac

**This Jupyter Notebook is intended to:**<br>
1) Read glider data frome ERDDAP <br>
2) Read in AD2CP data  <br>
3) Pre-porcess and Least squares linear inversion on ADCP velocities referenced to true ocean velocity through a depth averaged urrent constraint <br>
4) Save output from each segment<br>

*Details/comments on what the functions are actually doing in the source code*

In [1]:
# Imports
import scipy.interpolate as interp
from scipy.sparse.linalg import lsqr
import scipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import netCDF4 as nc
import math
import datetime
import xarray as xr
import matplotlib.dates as mdates
import dask.array as da
from erddapy import ERDDAP
from netCDF4 import Dataset
import gsw
import cmocean.cm as cmo
import sys
from datetime import datetime


## To import functions from Slocum-AD2CP GitHub repository, make this path the path to where the repo exists locally
sys.path.insert(0,'../src/analysis/')
sys.path.insert(0,'../src/data/')
from make_dataset import correct_sound_speed, beam_true_depth, cell_vert, binmap_adcp, beam2enu, inversion, qaqc_pre_coord_transform, qaqc_post_coord_transform, mag_var_correction
from analysis import get_erddap_dataset



import importlib
import NicolaiFunctions
importlib.reload(NicolaiFunctions)
from NicolaiFunctions import calcAHRS as calcAHRS_NI
from NicolaiFunctions import beam2enu as beam2enu_NI
from NicolaiFunctions import correct_ad2cp_heading
from NicolaiFunctions import inversion as inversion_NI

## Inputs

In [ ]:
#dataset id for erddap call - delayed mode, all glider flight data
ds_id = 'ru36-20230525T1748-trajectory-raw-delayed'

#dataset id for erddap call - real time mode, has magnetic declination (m_gps_mag_var)
ds_id2 = 'ru36-20230525T1748-trajectory-raw-rt'

#path to where you have your ad2cp netcdfs
path='../Passengers/AD2CP_data_leg2/pass2_0000_netcdf_100000kb_partition_ncs_no_header/'

#ad2cp data mode you want to process, Burst or Average *** CASE SENSITIVE ***
ad2cp_mode = 'Burst'

#inputs to the linear inversion
dz=10
wDAC = 5
wSmoothness = 1

#path for output csvs from linear inversion processing
csv_path="../Passengers/AD2CP_data_leg2/ad2cp_csv/"

# in the steps 3-10 for loop a segment number will be appended to this name
csv_name="RU36_2023_AD2CP_Processed_Segment_"

#path and filename for the processed netcdf
nc_fname='../Passengers/AD2CP_data_leg2/processed_nc/Passengers2_ad2cp_20230525_to_20230614_magvar.nc'

#metadata string for the processed netcdf
metadata_string="RU36 Nortek AD2CP least-squares inversion velocity profile dataset \nPassengers Project \nhttp://slocum-data.marine.rutgers.edu/erddap/tabledap/ru36-20230525T1748-trajectory-raw-delayed.html"

## Step 1: Load Glider Data

In [ ]:
## Load flight data with ds_id
variables = ['depth', 'latitude', 'longitude', 'time', 'sci_water_temp', 'sci_water_cond','source_file', 'm_water_vx', 'm_water_vy', 'm_heading']
gdf = get_erddap_dataset(ds_id, server='http://slocum-data.marine.rutgers.edu/erddap', variables = variables, filetype='dataframe')
gdf.columns = variables

## Great way to find start and end times!!
start_times = gdf.groupby('source_file').first().time.values
end_times   = gdf.groupby('source_file').last().time.values
## Remove time zone for slicing ad2cp times
start_times2 = pd.to_datetime(start_times).tz_localize(None)
end_times2 = pd.to_datetime(end_times).tz_localize(None)



## Load flight data with ds_id2, need magnetic declination to rotate depth averaged current later on
variables2 = ['time','m_gps_mag_var']
gdf2 = get_erddap_dataset(ds_id2, server='http://slocum-data.marine.rutgers.edu/erddap', variables = variables2, filetype='dataframe')
gdf2.columns = variables2

#merge both delayed and real-time datasets
gdf=gdf.merge(gdf2,on='time',how='left')
gdf

## Step 2: Load in AD2CP Dataset

In [ ]:
files = np.sort(glob.glob(path+'*.nc'))

tot_ad2cp = xr.open_mfdataset(files,group='Data/'+ad2cp_mode+'/', concat_dim="time", combine="nested")
## Because files are not necessarily read in time order with above line
tot_ad2cp = tot_ad2cp.sortby('time')
## I think this is a documented bug in the xarray open_mfdataset function for grouped NetCDFs
config = xr.open_dataset(files[0],group='Config')
## So, just assigning the global attributes from the first file to the combined file
tot_ad2cp = tot_ad2cp.assign_attrs(config.attrs)
tot_ad2cp = tot_ad2cp.rename({'Velocity Range':'VelocityRange','Correlation Range':'CorrelationRange','Amplitude Range':'AmplitudeRange'})
# 2 House-keeping steps
# 1) Roll shifted 180 for some reason
tot_ad2cp['Roll'] = tot_ad2cp['Roll'] 
# 2) Surface depth is 10 meters. Needed for at least for the 2020 and both 2021 deployments, per conversations with Sven from Nortek.
tot_ad2cp['Pressure'] = tot_ad2cp['Pressure']-10
# Put time on x-dimension
tot_ad2cp = tot_ad2cp.transpose()

# !!!! specific to this deployment because it was recovered 6/14 , need to subset time !!!!
tot_ad2cp=tot_ad2cp.sel(time=slice('2023-05-25', '2023-06-14'))
tot_ad2cp

## Steps 3-10: Big loop to process velocity data and save output

In [ ]:
for x in range(0,len(start_times)):

    ## Subset glider df to one segment
    subsetgdf = gdf[(gdf.time >= start_times[x]) & (gdf.time <= end_times[x])]
    
    ind         = np.argwhere(np.isnan(subsetgdf.m_water_vx).ravel()==False).flatten()
    
    if len(ind)>0:
        ## Pull out last non-NaN lat/lon
        # ind1         = np.argwhere(np.isnan(subsetgdf.longitude).ravel()==False).flatten()
#         vx          = subsetgdf.m_water_vx.iloc[ind1[-1]]
#         vy          = subsetgdf.m_water_vy.iloc[ind1[-1]]
#         vx_start_lon = subsetgdf.longitude.iloc[ind1[0]]
#         vx_start_lat = subsetgdf.latitude.iloc[ind1[0]]
#         vx_end_lon   = subsetgdf.longitude.iloc[ind1[-1]]
#         vx_end_lat   = subsetgdf.latitude.iloc[ind1[-1]]
        
        
        vx          = subsetgdf.m_water_vx.iloc[ind[-1]]
        vy          = subsetgdf.m_water_vy.iloc[ind[-1]]
        mag_var     = subsetgdf.m_gps_mag_var.iloc[ind[-1]]
        vx_start_lon = subsetgdf.longitude.iloc[ind[0]]
        vx_start_lat = subsetgdf.latitude.iloc[ind[0]]
        vx_end_lon   = subsetgdf.longitude.iloc[ind[-1]]
        vx_end_lat   = subsetgdf.latitude.iloc[ind[-1]]
        vx_start_tm = subsetgdf.time.iloc[0]
        vx_end_tm   = subsetgdf.time.iloc[-1]
        
    if subsetgdf.depth.max() < 10:
            continue
            
            
    ad2cp_time_ind = np.where((tot_ad2cp.time.values >= start_times2[x]) & (tot_ad2cp.time.values <= end_times2[x]))[0]
    if len(ad2cp_time_ind) > 0:

        subset_ad2cp = tot_ad2cp.sel(time= slice(tot_ad2cp.time.values[ad2cp_time_ind[0]],tot_ad2cp.time.values[ad2cp_time_ind[-1]]))

        # Just check if there is still data after the subsetting 
        if len(subset_ad2cp.time) > 0:
            
            # Pre -process data before inversion
            subset_ad2cp = correct_sound_speed(subset_ad2cp)
            subset_ad2cp = qaqc_pre_coord_transform(subset_ad2cp, corr_threshold = 50, max_amplitude = 75)
            subset_ad2cp = beam_true_depth(subset_ad2cp)
            subset_ad2cp = binmap_adcp(subset_ad2cp)
            # correct heading, vx, and vy with m_gps_mag_var
            heading_cor,vx_cor,vy_cor=mag_var_correction(subset_ad2cp['Heading'].values,vx,vy,mag_var)

            if ad2cp_mode == 'Average':
                subset_ad2cp.attrs['beam2xyz'] = subset_ad2cp.avg_beam2xyz.reshape(4,4)

            elif ad2cp_mode=='Burst':  
                subset_ad2cp.attrs['beam2xyz'] = subset_ad2cp.burst_beam2xyz.reshape(4,4)
            # RotMatrix=calcAHRS_NI(subset_ad2cp['Heading'].values,subset_ad2cp['Roll'].values,subset_ad2cp['Pitch'].values)
            RotMatrix=calcAHRS_NI(heading_cor,subset_ad2cp['Roll'].values,subset_ad2cp['Pitch'].values)
            # Get your conditions
            subset_ad2cp['AHRSRotationMatrix'] = (('x','time'), RotMatrix)
            subset_ad2cp = beam2enu_NI(subset_ad2cp)

            subset_ad2cp = qaqc_post_coord_transform(subset_ad2cp, high_velocity_threshold=0.75, surface_depth_to_filter = 5)
           
            # ## Now ready for inversion!

            O_ls, G_ls, bin_new, obs_per_bin = inversion_NI(subset_ad2cp.UVelocity.values,subset_ad2cp.VVelocity.values,dz,vx_cor,vy_cor,subset_ad2cp['VelocityRange'].values,subset_ad2cp['Pressure'].values, wDAC, wSmoothness)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Inversion", x ,"out of",len(start_times),"at" ,now)

            ###############################################
            #             Save master dataset             #
            ###############################################
            fname = csv_path+csv_name+str(x)+".csv".format(dz,x)

            ## Make into a dataframe to save as a CSV
            d = {'inversion_u': np.real(O_ls), 'inversion_v': np.imag(O_ls), "inversion_depth": bin_new,
                 "start_lon": np.tile(vx_start_lon,len(bin_new)), "start_lat": np.tile(vx_start_lat,len(bin_new)),
                 "end_lon": np.tile(vx_end_lon,len(bin_new)), "end_lat": np.tile(vx_end_lat,len(bin_new)),
                 "start_tm": np.tile(vx_start_tm, len(bin_new)), "end_tm": np.tile(vx_end_tm, len(bin_new)),
                 "obs_per_bin": obs_per_bin}


            df = pd.DataFrame(data=d)
            df.to_csv(fname,index=False) 
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Writing Data", x ,"out of",len(start_times),"at" ,now)
            del subset_ad2cp
            
        else:
            del subset_ad2cp
            

## Load in Processed AD2CP Data and Export NetCDF

In [ ]:
files = np.sort(glob.glob(csv_path+'*.csv'))
df = pd.concat(map(pd.read_csv, files))

inversion_dz = np.diff(df.inversion_depth)[0].astype(int)
inversion_depth         = np.arange(np.min(df.inversion_depth),np.max(df.inversion_depth)+inversion_dz,inversion_dz)
inversion_time          = np.empty(len(files))
inversion_time[:]       = np.NaN
inversion_time          = inversion_time.astype(pd.Timestamp)
inversion_start_time    = np.empty(len(files))
inversion_start_time[:] = np.NaN
inversion_start_time    = inversion_time.astype(pd.Timestamp)
inversion_end_time      = np.empty(len(files))
inversion_end_time[:]   = np.NaN
inversion_end_time      = inversion_time.astype(pd.Timestamp)
inversion_lat           = np.empty(len(files))
inversion_lat[:]        = np.NaN
inversion_lon           = np.empty(len(files))
inversion_lon[:]        = np.NaN
inversion_start_lat     = np.empty(len(files))
inversion_start_lat[:]  = np.NaN
inversion_start_lon     = np.empty(len(files))
inversion_start_lon[:]  = np.NaN
inversion_end_lat       = np.empty(len(files))
inversion_end_lat[:]    = np.NaN
inversion_end_lon       = np.empty(len(files))
inversion_end_lon[:]    = np.NaN
u_grid = np.empty((len(inversion_depth),len(files)))
u_grid[:] = np.NaN
v_grid = np.empty((len(inversion_depth),len(files)))
v_grid[:] = np.NaN


## Loop through by file, load in each file
for x in np.arange(0,len(files)):
    
    df = pd.read_csv(files[x])
    
    u_grid[np.arange(0,len(df.inversion_u.values)),x] = df.inversion_u.values
    v_grid[np.arange(0,len(df.inversion_v.values)),x] = df.inversion_v.values
    
    inversion_start_time[x] = pd.to_datetime(df.start_tm[0]).tz_localize(None)
    inversion_end_time[x] = pd.to_datetime(df.end_tm[0]).tz_localize(None)
    mid_time = inversion_end_time[x]-inversion_start_time[x]
    
    inversion_time[x] = inversion_start_time[x]+mid_time

    inversion_start_lat[x] = df.start_lat[0]
    inversion_start_lon[x] = df.start_lon[0]
    inversion_end_lat[x]   = df.end_lat[0]
    inversion_end_lon[x]   = df.end_lon[0]
    ## Lat/lon mid point
    inversion_lat[x]   = (df.start_lat[0]+df.end_lat[0])/2
    inversion_lon[x]   = (df.start_lon[0]+df.end_lon[0])/2


    
## Now stuff into an organized xarray dataset    
ds = xr.Dataset(
    data_vars=dict(
        u_grid     = (["depth", "time"], u_grid),
        v_grid     = (["depth", "time"], v_grid),
        latitude   = (["time"], inversion_lat),
        longitude  = (["time"], inversion_lon),
        start_lat  = (["time"], inversion_start_lat),
        start_lon  = (["time"], inversion_start_lon),
        end_lat    = (["time"], inversion_end_lat),
        end_lon    = (["time"], inversion_end_lon),
        start_time = (["time"], inversion_start_time),
        end_time   = (["time"], inversion_end_time)
    ),
    coords=dict(
        time  = inversion_time,
        depth = inversion_depth
    ),
    attrs=dict(description=metadata_string),
)

## Sort by time because files may not have in read in chronological order
ds = ds.sortby(ds.time)
#save output to netcdf
ds.to_netcdf(nc_fname)
ds